# Assignment

In this assignment we will train a convolutional neural network (CNN) to detect the normal kidney on a slice-by-slice basis from abdominal CT exams. 

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 89 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about he KiTS Challenge can be found here: https://kits21.kits-challenge.org/. In this exercise, we will use this dataset to derive a model for slice-by-slice kidney detection. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset

In [3]:
# --- Download dataset
datasets.download(name='ct/kits-128')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='ct/kits-128', keyword='128*glb')

[ 2022-04-22 17:09:15 ] [====================] 100.000% : Extracting archive (0000418 / 0000418) 

# Training

In this assignment we will train a basic convolutional neural network to predict the correct imaging series protocol on prostate MRI. At minumum you must include one of the following modern CNN architecture motifs techniques covered in the tutorial:

* residual function (with preactivation blocks)
* Inception module
* squeeze-and-excitation module

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define backbone model

Feel free to use the `lambda` helper functions as demonstrated in the tutorial.

In [4]:
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding'    : 'same',
    'kernel_initializer' : 'he_normal'
}

#lambda

conv = lambda x, filters, strides: layers.Conv3D(filters = filters , strides = strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides = (1, 2, 2))))

In [5]:
def s_e(layer):

  sqz = layers.AveragePooling3D((1, layer.shape[2], layer.shape[3]))(layer)
  cha = int(layer.shape[-1]/4)
  exc = layers.Conv3D(filters = cha, kernel_size = 1, activation = 'relu')(sqz)

  sca = layers.Conv3D(filters = layer.shape[-1], kernel_size = 1, activation = 'sigmoid')(exc)
  return layer * sca

In [6]:
# --- Define input
x = Input(shape=(None, 128, 128, 1), dtype='float32')

# --- Define model
l1 = conv1(32,  x)
l1 = s_e(l1)

l2 = conv1(48,  conv2(48,  l1))
l2 = s_e(l2)

l3 = conv1(64,  conv2(64,  l2))
l3 = s_e(l3)

l4 = conv1(80,  conv2(80,  l3))
l4 = s_e(l4)

l5 = conv1(96,  conv2(96,  l4))
l5 = s_e(l5)

l6 = conv1(128, conv2(128, l5))
l6 = s_e(l6)


n0, n1, c = l6.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l6)
# --- Define logits
logits = layers.Conv3D(filters = 2, kernel_size = 1)(f0)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

In [7]:
logits

<KerasTensor: shape=(None, None, 1, 1, 2) dtype=float32 (created by layer 'conv3d_23')>

### Define training model

In [8]:
# --- Define inputs
inputs = {
    'dat': Input(shape = (1, 128, 128, 1), name = 'dat'),
    'lbl': Input(shape = (1, 1, 1, 1,), name = 'lbl')}

# --- Define model
logits = backbone(inputs['dat'])

# --- Define loss
sce = losses.SparseCategoricalCrossentropy(from_logits=True)
loss = sce(y_true=inputs['lbl'], y_pred=logits)
# --- Define metric
acc = metrics.sparse_categorical_accuracy(y_true=inputs['lbl'], y_pred=logits)

Now, we are ready to create the `training` model and add the corresponding loss and accuracy tensors:

In [9]:
# --- Create model
training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'acc': acc})

# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(acc, name = 'acc')

### Compiling

Once the `training` model has been created, use the following to define an optimizer and compile:

In [10]:
# --- Define optimizer 
optimizer = optimizers.Adam(learning_rate = 2e-4)

# --- Compile model
training.compile(optimizer = optimizer)

The model is now compiled and ready for training!

### In-memory data

To speed up training, consider loading all your model data into RAM memory:

In [11]:
# --- Load data into memory for faster training
client.load_data_in_memory()

[ 2022-04-22 17:09:25 ] [====================] 100.000% : Iterating | 000203    

### Train the model

In [17]:
training.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=30,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=5)

Epoch 1/30
100/100 [==============================] - 2s 23ms/step - loss: 0.1556 - acc: 0.9337
Epoch 2/30
100/100 [==============================] - 2s 23ms/step - loss: 0.1136 - acc: 0.9500
Epoch 3/30
100/100 [==============================] - 2s 23ms/step - loss: 0.1361 - acc: 0.9550
Epoch 4/30
100/100 [==============================] - 2s 22ms/step - loss: 0.1377 - acc: 0.9500
Epoch 5/30
100/100 [==============================] - 3s 34ms/step - loss: 0.1285 - acc: 0.9463 - val_loss: 0.2372 - val_acc: 0.9125
Epoch 6/30
100/100 [==============================] - 2s 23ms/step - loss: 0.1172 - acc: 0.9525
Epoch 7/30
100/100 [==============================] - 2s 22ms/step - loss: 0.1254 - acc: 0.9388
Epoch 8/30
100/100 [==============================] - 2s 23ms/step - loss: 0.1294 - acc: 0.9513
Epoch 9/30
100/100 [==============================] - 2s 23ms/step - loss: 0.1313 - acc: 0.9400
Epoch 10/30
100/100 [==============================] - 3s 35ms/step - loss: 0.1019 - acc: 0.9625 - 

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

**Important**: In this assignment, you must obtain >80% performance accuracy to recieve full credit. Accuracy is determined on slice-by-slice performance accuracy as demonstrated in the tutorial; please refer to tutorial materials if you have questions.

In [18]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

**Note**: this cell is used only to check for model performance prior to submission. It will not be graded. Once submitted, your model will be benchmarked against the (same) validation cohort to determine final algorithm performance and grade. If your evaluation code above is correct the algorithm accuracy should match and you can be confident that you will recieve full credit for the assignment. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [19]:
test_train, test_valid = client.create_generators(test=True, expand=True)

trues = []
preds = []

for x, _ in test_valid:
    
    # --- Predict
    logits = backbone.predict(x['dat'])

    # --- Argmax
    pred = np.squeeze(np.argmax(logits, axis=-1))

    trues.append(x['lbl'].ravel())
    preds.append(pred.ravel())

trues = np.concatenate(trues)
preds = np.concatenate(preds)

[ 2022-04-22 17:11:42 ] [====================] 100.000% : Iterating | 000041    

In [20]:
# --- Create *.csv
df = pd.DataFrame(index=np.arange(preds.size))

# --- Define columns
df['true'] = trues
df['pred'] = preds
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())
                              
# --- Serialize *.csv
df.to_csv('./wjhan_results.csv')

0.9324148687288797


# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [21]:
# --- Serialize a model
backbone.save('./wjhan_model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.